In [82]:
# Importing important libraries
import requests as r
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import os ,sys

In [83]:
#Getting all links from the web series page
link = 'https://www.imdb.com/list/ls525109030/?ref_=hm_edcft_c_geo_2_cta'
data = r.get(link).text
soup = bs(data,'lxml')
title = soup.find('h3',attrs={'class':'lister-item-header'})

In [84]:
#Function for extracting links from the h3 tags
def extract_data_from_page(soup):
    h3_tags = soup.find_all('h3')
    data_list = []
    for h3_tag in h3_tags:
        anchor_tag = h3_tag.a
        if anchor_tag:
            title = anchor_tag.text
            link = anchor_tag['href']
            data_dict = {
                "title": title,
                "link": link
            }
            data_list.append(data_dict)
    return data_list

In [85]:
df_link = extract_data_from_page(soup)
web_series = pd.DataFrame(df_link)

In [86]:
#storing links after adding remaining part of link
links = []
for i in web_series['link']:
    a = f"https://www.imdb.com{i}reviews?spoiler=hide&sort=curated&dir=desc&ratingFilter=0"
    links.append(a)

In [66]:
# checking if code working or not
all_reviews = []
for j in links:
    requests = r.get(j).text
    soups = bs(requests, 'lxml')
    reviews = soups.find_all('div', attrs={'class': 'text show-more__control'})

    for review in reviews:
        rev = review.text.replace('\n', "")
        all_reviews.append(rev)

In [67]:
len(all_reviews)

250

In [92]:
#Automation for getting all reviews
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup
import time

# Use the webdriver.Chrome() constructor without the executable_path argument
driver = webdriver.Chrome()

all_reviews = []

for link in links:
    link = link.strip()  # Remove any leading/trailing whitespaces

    if not link.startswith('http'):  # Check if the link is a valid URL
        print(f"Invalid URL: {link}. Skipping.")
        continue

    driver.get(link)

    try:
        # Create an explicit wait to wait for the "Next" button to be present
        wait = WebDriverWait(driver, 20)

        # Counter to keep track of retries
        retry_count = 0
        max_retries = 2  # Maximum number of retries

        while True:
            try:
                # Find the "Next" button and wait for it to be clickable
                next_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'ipl-load-more__button')))

                # Scroll to the "Next" button and click it
                driver.execute_script("arguments[0].scrollIntoView();", next_button)
                next_button.click()

                # Wait for the new content to load (adjust the waiting time as needed)
                time.sleep(5)

            except NoSuchElementException:
                # If the "Next" button is not found, break the loop
                print("No more 'Next' button found. Exiting.")
                break

            except TimeoutException:
                # If the button is not clickable within the timeout, retry up to max_retries times
                retry_count += 1
                if retry_count > max_retries:
                    print(f"Max retries reached for waiting on the 'Next' button. Exiting.")
                    break
                print("Timeout while waiting for the 'Next' button to be clickable. Retrying...")
                continue

            except Exception as e:
                # Catching other unexpected exceptions and printing the error message for debugging
                print(f"An exception occurred: {e}")
                break

        # Extract the HTML content after the dynamic content has been loaded
        data = driver.page_source

        # Create BeautifulSoup object for parsing
        soup = BeautifulSoup(data, 'html.parser')

        # Now you can work with the 'soup' object to extract the desired data
        reviews = soup.find_all('div', attrs={'class': 'text show-more__control'})

        for review in reviews:
            rev = review.text.replace('\n', "")
            all_reviews.append(rev)

    except Exception as e:
        print(f"An exception occurred while scraping reviews from {link}: {e}")
        # If there's an error, continue to the next link
        continue

# Print the collected reviews for each link
for i, review in enumerate(all_reviews, start=1):
    print(f"Review {i}: {review}")

# Close the WebDriver
driver.quit()

Timeout while waiting for the 'Next' button to be clickable. Retrying...
Timeout while waiting for the 'Next' button to be clickable. Retrying...
Max retries reached for waiting on the 'Next' button. Exiting.
Timeout while waiting for the 'Next' button to be clickable. Retrying...
Timeout while waiting for the 'Next' button to be clickable. Retrying...
Max retries reached for waiting on the 'Next' button. Exiting.
Timeout while waiting for the 'Next' button to be clickable. Retrying...
Timeout while waiting for the 'Next' button to be clickable. Retrying...
Max retries reached for waiting on the 'Next' button. Exiting.
Timeout while waiting for the 'Next' button to be clickable. Retrying...
Timeout while waiting for the 'Next' button to be clickable. Retrying...
Max retries reached for waiting on the 'Next' button. Exiting.
Timeout while waiting for the 'Next' button to be clickable. Retrying...
Timeout while waiting for the 'Next' button to be clickable. Retrying...
Max retries reache

In [104]:
web_series_reviews = pd.DataFrame(all_reviews)

In [105]:
web_series_reviews.columns = ['reviews']

In [122]:
web_series_reviews.to_csv('web_series_reviews.csv')

In [112]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

In [118]:
from tqdm import tqdm
import re
import contractions
from nltk.corpus import stopwords

# Download the stopwords if not already downloaded
import nltk
nltk.download('stopwords')

stopwords = set(stopwords.words('english'))

preprocessed_reviews = []

# tqdm is for printing the status bar
for sentence in tqdm(web_series_reviews['reviews'].values):
    sentence = re.sub(r"http\S+", "", sentence)
    sentence = contractions.fix(sentence)
    sentence = re.sub("S\*d\S*",' ', sentence).strip()
    sentence = re.sub('[^A-Za-z]+',' ', sentence)
    sentence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentence.strip())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\deepa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|████████████████████████████████████████████████████████████████████████████| 4808/4808 [00:01<00:00, 4238.09it/s]


In [120]:
from textblob import TextBlob

def get_sentiment(review):
    blob = TextBlob(review)
    polarity = blob.sentiment.polarity
    # If polarity is greater than 0, the review is positive. Otherwise, it's negative.
    return 'positive' if polarity > 0 else 'negative'

positive_reviews = []
negative_reviews = []

for review in preprocessed_reviews:
    sentiment = get_sentiment(review)
    if sentiment == 'positive':
        positive_reviews.append(review)
    else:
        negative_reviews.append(review)
